Mounted at /content/drive


<ipython-input-1-c99e6125c82f>:6: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv("/content/drive/MyDrive/Min/all_min.csv")


In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
drive.mount("/content/drive")
raw_data = pd.read_csv("/content/drive/MyDrive/Min/all_min.csv", index_col = 0)
print(raw_data.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                Market Arrival Date  Arrivals (Tonnes) Variety  \
0  ACF Agro Marketing    26-04-2024              91.85  Cotton   
1  ACF Agro Marketing    27-04-2024              79.02  Cotton   
2  ACF Agro Marketing    30-04-2024              80.62  Cotton   
3        Akhadabalapur   01-04-2024              14.60  Cotton   
4                Akola   01-04-2024               3.50  Cotton   

  Minimum Price(Rs./Quintal)  
0                       6400  
1                       6700  
2                       6800  
3                       6700  
4                       7425  


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
raw_data = raw_data[~(raw_data["Minimum Price(Rs./Quintal)"] == "NR")]
raw_data["Minimum Price(Rs./Quintal)"] = raw_data["Minimum Price(Rs./Quintal)"].astype(dtype = np.float64)
raw_data["Minimum Price(Rs./Quintal)"] = raw_data["Minimum Price(Rs./Quintal)"].astype(dtype = np.int64)

def standardize_date(date_str):

  if isinstance(date_str, str):
    if '-' in date_str and len(date_str.split('-')[0]) == 2:
      return pd.to_datetime(date_str, format='%d-%m-%Y')
    return pd.to_datetime(date_str)
  return pd.to_datetime(date_str)

raw_data['Arrival Date'] = raw_data['Arrival Date'].apply(standardize_date)
raw_data = raw_data.sort_values(['Market', 'Variety', 'Arrival Date'])

In [ ]:
print(raw_data.describe(include = 'all'))

        Market                   Arrival Date  Arrivals (Tonnes)   Variety  \
count   165841                         165841      165841.000000    165841   
unique     320                            NaN                NaN        10   
top     Mumbai                            NaN                NaN  Soyabean   
freq      2591                            NaN                NaN     44621   
mean       NaN  2024-01-12 09:24:37.827557632          49.294928       NaN   
min        NaN            2023-01-01 00:00:00           0.010000       NaN   
25%        NaN            2023-07-15 00:00:00           2.000000       NaN   
50%        NaN            2024-01-04 00:00:00           7.200000       NaN   
75%        NaN            2024-07-12 00:00:00          33.000000       NaN   
max        NaN            2024-12-31 00:00:00        6426.000000       NaN   
std        NaN                            NaN         148.832694       NaN   

        Minimum Price(Rs./Quintal)  
count                16584

In [ ]:
#raw_data = raw_data["Minimum Price(Rs./Quintal)"].astype(dtype = np.float64)
raw_data = raw_data[~(raw_data["Minimum Price(Rs./Quintal)"] == "NR")]

In [ ]:
raw_data["Minimum Price(Rs./Quintal)"] = raw_data["Minimum Price(Rs./Quintal)"].astype(dtype = np.float64)
raw_data["Minimum Price(Rs./Quintal)"] = raw_data["Minimum Price(Rs./Quintal)"].astype(dtype = np.int64)

In [ ]:
print(raw_data.describe())

                        Arrival Date  Arrivals (Tonnes)  \
count                         165841      165841.000000   
mean   2024-01-12 09:24:37.827557632          49.294928   
min              2023-01-01 00:00:00           0.010000   
25%              2023-07-15 00:00:00           2.000000   
50%              2024-01-04 00:00:00           7.200000   
75%              2024-07-12 00:00:00          33.000000   
max              2024-12-31 00:00:00        6426.000000   
std                              NaN         148.832694   

       Minimum Price(Rs./Quintal)  
count               165841.000000  
mean                  3215.309947  
min                      2.000000  
25%                   2100.000000  
50%                   2530.000000  
75%                   4200.000000  
max                 110000.000000  
std                   1513.724291  


In [ ]:
# Sort the data by Market, Variety, and Arrival Date
raw_data = raw_data.sort_values(['Market', 'Variety', 'Arrival Date'])

In [ ]:
print(raw_data.head())

                    Market Arrival Date  Arrivals (Tonnes) Variety  \
8550   ACF Agro Marketing    2023-01-07              87.95  Cotton   
10488  ACF Agro Marketing    2023-01-08             132.21  Cotton   
8551   ACF Agro Marketing    2023-02-07             229.18  Cotton   
8552   ACF Agro Marketing    2023-03-07             185.90  Cotton   
8553   ACF Agro Marketing    2023-04-07              98.96  Cotton   

       Minimum Price(Rs./Quintal)  
8550                         6700  
10488                        6700  
8551                         6800  
8552                         6800  
8553                         6800  


In [ ]:
raw_data['Year'] = raw_data['Arrival Date'].dt.year
raw_data['Month'] = raw_data['Arrival Date'].dt.month
raw_data['Day'] = raw_data['Arrival Date'].dt.day
raw_data['Quarter'] = raw_data['Arrival Date'].dt.quarter
raw_data['DayOfWeek'] = raw_data['Arrival Date'].dt.dayofweek

In [ ]:
for lag in range(1, 4):
    raw_data[f'Lag_{lag}'] = raw_data.groupby(['Market', 'Variety'])['Minimum Price(Rs./Quintal)'].shift(lag)

raw_data = raw_data.dropna()

In [ ]:
rolling_windows = {
    'Rolling_Mean_7': 7,
    'Rolling_Mean_14': 14,
    'Rolling_Mean_30': 30,
    'Rolling_Std_7': 7,
    'Rolling_Std_14': 14,
    'Rolling_Std_30': 30,
    'Arrival_Rolling_mean_3': 3
}

for window_name, window_size in rolling_windows.items():
    if 'Mean' in window_name:
        raw_data[window_name] = raw_data.groupby(['Market', 'Variety'])['Minimum Price(Rs./Quintal)'].transform(
            lambda x: x.rolling(window=window_size, min_periods=1).mean()
        )
    elif 'Std' in window_name:
        raw_data[window_name] = raw_data.groupby(['Market', 'Variety'])['Minimum Price(Rs./Quintal)'].transform(
            lambda x: x.rolling(window=window_size, min_periods=1).std()
        )
    else:
        raw_data[window_name] = raw_data.groupby(['Market', 'Variety'])['Arrivals (Tonnes)'].transform(
            lambda x: x.rolling(window=window_size, min_periods=1).mean()
        )

In [ ]:
raw_data['Market'] = raw_data['Market'].astype('category')
raw_data['Variety'] = raw_data['Variety'].astype('category')

In [ ]:
print(raw_data.describe(include = 'all'))

        Market                   Arrival Date  Arrivals (Tonnes)   Variety  \
count   162411                         162411      162411.000000    162411   
unique     309                            NaN                NaN        10   
top     Mumbai                            NaN                NaN  Soyabean   
freq      2572                            NaN                NaN     43900   
mean       NaN  2024-01-18 02:29:51.056024576          49.810078       NaN   
min        NaN            2023-01-03 00:00:00           0.010000       NaN   
25%        NaN            2023-07-24 00:00:00           2.000000       NaN   
50%        NaN            2024-01-11 00:00:00           7.500000       NaN   
75%        NaN            2024-07-18 00:00:00          33.850000       NaN   
max        NaN            2024-12-31 00:00:00        6426.000000       NaN   
std        NaN                            NaN         149.903025       NaN   

        Minimum Price(Rs./Quintal)           Year          Mont

In [ ]:
raw_data.to_csv("/content/drive/MyDrive/EngineeredData.csv")

In [ ]:
X = raw_data.drop(columns=['Minimum Price(Rs./Quintal)', 'Arrival Date'])
y = raw_data['Minimum Price(Rs./Quintal)']

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import xgboost as xgb

model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    enable_categorical=True
)

model.fit(X_train, y_train)
y_pred_val = model.predict(X_val)
y_pred_train = model.predict(X_train)

from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
print(mean_absolute_percentage_error(y_val, y_pred_val))
mse = mean_squared_error(y_val, y_pred_val)
print(mean_absolute_error(y_val, y_pred_val))

print(mean_absolute_percentage_error(y_train, y_pred_train))

0.06057712435722351
158.7999725341797
0.06291460990905762


In [ ]:
def predict_future(model, raw_data, n_future_steps, target_market, target_variety):

    historical_data = raw_data[
        (raw_data['Market'] == target_market) &
        (raw_data['Variety'] == target_variety)
    ]


    initial_data = historical_data.iloc[[-1]].drop(columns=['Minimum Price(Rs./Quintal)', 'Arrival Date'])
    initial_data['Market'] = initial_data['Market'].astype('category')
    initial_data['Variety'] = initial_data['Variety'].astype('category')

    historical_prices = historical_data['Minimum Price(Rs./Quintal)']

    predictions = []
    current_data = initial_data.copy()

    rolling_window_7 = historical_prices.tail(7).tolist()
    rolling_window_14 = historical_prices.tail(14).tolist()
    rolling_window_30 = historical_prices.tail(30).tolist()

    rolling_std_window_7 = historical_prices.tail(7).tolist()
    rolling_std_window_14 = historical_prices.tail(14).tolist()
    rolling_std_window_30 = historical_prices.tail(30).tolist()

    arrival_rolling_window_3 = historical_data['Arrivals (Tonnes)'].tail(3).tolist()

    latest_prices = historical_prices.tail(3).tolist()
    current_data['Lag_1'] = latest_prices[-1]
    current_data['Lag_2'] = latest_prices[-2]
    current_data['Lag_3'] = latest_prices[-3]

    for _ in range(n_future_steps):

        next_arrival = np.mean(arrival_rolling_window_3)
        current_data['Arrivals (Tonnes)'] = next_arrival

        arrival_rolling_window_3.append(next_arrival)
        if len(arrival_rolling_window_3) > 3:
            arrival_rolling_window_3.pop(0)
        current_data['Arrival_Rolling_mean_3'] = np.mean(arrival_rolling_window_3)

        next_price = model.predict(current_data)[0]
        predictions.append(next_price)

        current_data['Lag_3'] = current_data['Lag_2']
        current_data['Lag_2'] = current_data['Lag_1']
        current_data['Lag_1'] = next_price

        rolling_window_7.append(next_price)
        rolling_window_14.append(next_price)
        rolling_window_30.append(next_price)

        rolling_std_window_7.append(next_price)
        rolling_std_window_14.append(next_price)
        rolling_std_window_30.append(next_price)

        if len(rolling_window_7) > 7:
            rolling_window_7.pop(0)
        if len(rolling_window_14) > 14:
            rolling_window_14.pop(0)
        if len(rolling_window_30) > 30:
            rolling_window_30.pop(0)

        if len(rolling_std_window_7) > 7:
            rolling_std_window_7.pop(0)
        if len(rolling_std_window_14) > 14:
            rolling_std_window_14.pop(0)
        if len(rolling_std_window_30) > 30:
            rolling_std_window_30.pop(0)

        current_data['Rolling_Mean_7'] = np.mean(rolling_window_7)
        current_data['Rolling_Mean_14'] = np.mean(rolling_window_14)
        current_data['Rolling_Mean_30'] = np.mean(rolling_window_30)

        current_data['Rolling_Std_7'] = np.std(rolling_std_window_7)
        current_data['Rolling_Std_14'] = np.std(rolling_std_window_14)
        current_data['Rolling_Std_30'] = np.std(rolling_std_window_30)

        current_data['Market'] = current_data['Market'].astype('category')
        current_data['Variety'] = current_data['Variety'].astype('category')

    return predictions

In [ ]:
n_future_steps = 7
target_market = "Mumbai"
target_variety = "Sugar"

future_predictions = predict_future(model, raw_data, n_future_steps, target_market, target_variety)

print(f'Predicted prices for {target_market} ({target_variety}) for the next {n_future_steps} days:')
print(future_predictions)

Predicted prices for Mumbai (Sugar) for the next 7 days:
[2010.9707, 2010.9707, 2010.9707, 2012.7432, 2012.7432, 2012.7432, 2034.4119]


In [ ]:
import joblib
joblib.dump(model, '/content/drive/MyDrive/aissmspricemodel.pkl')

['/content/drive/MyDrive/aissmspricemodel.pkl']